In [93]:
import pandas as pd
import jieba
import numpy

In [94]:
df_news = pd.read_csv('news_content.csv')
df_news = df_news.dropna()
df_news.describe()

,Unnamed: 0,news_label
count,238.000000,238.000000
mean,118.500000,6.500000
std,68.848868,4.039624
min,0.000000,0.000000
25%,59.250000,3.000000
50%,118.500000,6.500000
75%,177.750000,10.000000
max,237.000000,13.000000


In [95]:
content = df_news.content.values.tolist()

In [96]:
content_S = []
for line in content:
    current_segment = jieba.lcut(line,cut_all=False)
    if len(current_segment) > 1 and current_segment != '\r\n': #换行符
        content_S.append(current_segment)

In [97]:
df_content=pd.DataFrame({'content_S':content_S})
df_content.head()

,content_S
0,"[行政院, 今, （, 7, ）, 通過, 《, 刑法, 》, 及, 《, 刑法, 施行, ..."
1,"[台北市, 議會, 工務委員會, ，, 今天, （, 7, 日, ）, 針對, 大, 巨蛋,..."
2,"[行政院, 長, 蘇貞昌, 出席, 今, （, 7, ）, 日, 行政院, 例行, 的, 「..."
3,"[罷韓, 公投, 將登場, ，, 而, 支持, 罷韓, 的, 台灣基, 進志工, 昨, （,..."
4,"[新台灣, 國策, 智庫, 今, （, 7, ）, 天, 公布, 了, 「, 520, 蔡,..."


In [98]:
stopwords=pd.read_csv("stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')
stopwords.head(20)

,stopword
0,!
1,""""
2,#
3,$
4,%
5,&
6,'
7,「
8,」
9,（


In [99]:
def drop_stopwords(contents,stopwords):
    contents_clean = []
    all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))
        contents_clean.append(line_clean)
    return contents_clean,all_words
    #print (contents_clean)
        

contents = df_content.content_S.values.tolist()    
stopwords = stopwords.stopword.values.tolist()
contents_clean,all_words = drop_stopwords(contents,stopwords)

#df_content.content_S.isin(stopwords.stopword)
#df_content=df_content[~df_content.content_S.isin(stopwords.stopword)]
#df_content.head()

In [100]:
df_content=pd.DataFrame({'contents_clean':contents_clean})
df_content.head()

,contents_clean
0,"[行政院, 通過, 刑法, 刑法, 施行, 法, 修正, 草案, 將, 妨害, 公務, 加重..."
1,"[台北市, 議會, 工務委員會, 針對, 巨蛋, 本體, 與, 國父, 紀念館, 之間, 的..."
2,"[行政院, 長, 蘇貞昌, 出席, 行政院, 例行, 紓, 困振興, 方案, 記者, 會, ..."
3,"[罷韓, 公投, 將登場, 支持, 罷韓, 台灣基, 進志工, 昨, 高雄, 街頭, 發放文..."
4,"[新台灣, 國策, 智庫, 天, 公布, 520, 蔡, 英文, 總統, 職, 總統, 內閣..."


In [101]:
df_all_words=pd.DataFrame({'all_words':all_words})
df_all_words.head()

,all_words
0,行政院
1,通過
2,刑法
3,刑法
4,施行


In [102]:
import jieba.analyse
index = 0
print (df_news['content'][index])
content_S_str = "".join(content_S[index])  
print('=====================================')
print ("  ".join(jieba.analyse.extract_tags(content_S_str, topK=5, withWeight=False)))

行政院今（7）通過《刑法》及《刑法施行法》修正草案，將妨害公務加重處罰，若駕車衝撞或以兇器對公務員施暴，刑度從最高3年有期徒刑，提高到5年有期徒刑，以維護公務員執法安全。法務部表示，近期駕車衝撞警員案頻傳，妨礙公務案件逐年攀升，增加警方或相關公務員執行職務時的風險，因此提出《刑法》修正案，以加強維護公務員人身安全。法務部檢察司司長林錦村表示，刑法第135條增訂「加重妨害公務罪」，行為人以駕駛動力交通工具或意圖供行使之用而攜帶兇器或其他危險物品，對公務員施強暴脅迫者，予以加重處罰，由現行最高3年有期徒刑提高到5年有期徒刑。另還修正刑法第136條公然聚眾的要件，行為人在公共場所聚集3人以上進行強暴脅迫行為，即給予處罰，以加強維護守法民眾出入安全。林錦村也說明，科技日新月異，新型科技犯罪也隱藏洗錢、組織暴力、恐嚇等犯罪，此次同時修正賭博罪章，增訂網路賭博罪，若以電信設備、電子通訊、網際網路等賭博者，予以處罰。此外，依照現行刑法規定，任何人若在假釋期間犯罪，不論輕重一律撤銷假釋，關回監獄，但不利於受刑人回歸社會，林錦村表示，修正刑法第78條，增定裁量機制，若在假釋期間犯罪，被法院判刑超過6個月，直接撤銷假釋；但如果犯罪是被判6個月以下，可以有裁量權，實際裁決依個案情形決定是否撤銷假釋。
假釋  刑法  處罰  公務員  犯罪


In [103]:
from gensim import corpora, models, similarities
import gensim
#http://radimrehurek.com/gensim/

In [104]:
#做映射，相当于词袋
dictionary = corpora.Dictionary(contents_clean)
corpus = [dictionary.doc2bow(sentence) for sentence in contents_clean]

In [105]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20) #类似Kmeans自己指定K值

In [106]:
print (lda.print_topic(1, topn=5))

0.021*" " + 0.009*"與" + 0.006*"為" + 0.006*"疫情" + 0.005*"後"


In [107]:
for topic in lda.print_topics(num_topics=20, num_words=5):
    print (topic[1])

0.008*" " + 0.007*"疫情" + 0.005*" " + 0.004*"讓" + 0.004*"與"
0.021*" " + 0.009*"與" + 0.006*"為" + 0.006*"疫情" + 0.005*"後"
0.010*" " + 0.006*"疫情" + 0.005*"對" + 0.005*"後" + 0.004*"為"
0.007*"我們" + 0.006*"後" + 0.005*"於" + 0.005*" " + 0.005*"讓"
0.007*"後" + 0.007*" " + 0.006*"與" + 0.006*" " + 0.005*"為"
0.041*" " + 0.006*"台灣" + 0.005*"疫情" + 0.005*"與" + 0.004*"後"
0.007*" " + 0.006*"為" + 0.005*"於" + 0.005*"後" + 0.004*" "
0.039*" " + 0.008*"疫情" + 0.008*"後" + 0.008*" " + 0.007*"與"
0.064*" " + 0.007*"疫情" + 0.005*" " + 0.005*"對" + 0.005*"與"
0.006*"讓" + 0.006*" " + 0.005*"為" + 0.005*"中" + 0.005*"與"
0.007*"對" + 0.006*" " + 0.005*"為" + 0.005*"我們" + 0.004*"於"
0.034*" " + 0.008*" " + 0.006*"疫情" + 0.006*"與" + 0.005*"台灣"
0.012*" " + 0.008*" " + 0.005*"後" + 0.004*"於" + 0.004*"為"
0.027*" " + 0.008*" " + 0.006*"與" + 0.006*"口罩" + 0.005*"後"
0.009*" " + 0.007*"與" + 0.005*"於" + 0.004*"對" + 0.004*"中國"
0.023*" " + 0.008*"口罩" + 0.006*"疫情" + 0.005*" " + 0.005*"於"
0.011*" " + 0.007*" " + 0.006*"後" + 0.004*"為" + 0.003*"讓"

In [108]:
import numpy as np
df_train=pd.DataFrame({'title':contents_clean,'label':df_news['news_label']})
sampler = np.random.permutation(230)
df_train = df_train.take(sampler)


In [109]:
df_train.describe()

,label
count,230.000000
mean,6.273913
std,3.919353
min,0.000000
25%,3.000000
50%,6.000000
75%,10.000000
max,13.000000


In [110]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train['title'].values, df_train['label'].values,test_size=0.3)

In [111]:
def not_empty(s):
    return s and s.strip()

for i in range(len(x_train)):
    x_train[i] = list(filter(not_empty, x_train[i]))
    
for i in range(len(x_test)):
    x_test[i] = list(filter(not_empty, x_test[i]))

In [112]:
words = []
test_words = []

for line_index in range(len(x_train)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        words.append(' '.join(x_train[line_index]))
    except:
        print (line_index,word_index)

for line_index in range(len(x_test)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        test_words.append(' '.join(x_test[line_index]))
    except:
         print (line_index,word_index)

In [113]:
len(test_words)

69

In [114]:
words[0]

'西洋 樂壇 流行 天后 瑪丹娜 Madonna 驚世駭俗 風格 反應 這次 新冠 肺炎 疫情 近來 頻頻 IG 發 影片 談新冠 病毒 先是 透露 位親 友染疫 離世 讓 難過 死去 活來 還說 病毒 一視 同仁 發文聲 稱 體內測 出新 冠 病毒 抗體 放話 將要 外出 吸 病毒 籠罩 空氣 後 來還 真的 跑 參加 友人 生日 趴 娜 姊 發文 確診 感染 新冠 肺炎 隨後更 語出 驚人 個禮 拜 前 表演 團隊 經中 鏢 娜 姊 IG 防疫 日記 引發 討論 圖 瑪丹娜 IG 娜 姊 日前 現身 攝影師 好友 生日 派 對 嗨 玩 還戲 謔 地將 生日蛋糕 取名 為 新冠 蛋糕 無視 社交 禁令 沒 想到 PO 文承認 確定 染上 新冠 肺炎 當你 抗體 篩 檢結果 呈 現陽性 代表 曾經 感染 過 病毒 很明顯 真的 感染 娜 姊還 約 個禮 拜 前 巴黎 巡演 結束 前 團隊 工作 人員 已經 生病 雖然 當時 每個 症狀 嚴重 為 流感 不過 強調 感謝 上帝 我們 現在 健康 希望 這些 說 明夠 知識 力量 編輯 楊穎軒 瑪丹娜 發文承認 感染 新冠 肺炎 圖 瑪丹娜 IG ※ NOWnews 今日 新聞 提醒您 因應 新冠 肺炎 疫情 疾管署 持續 加強 疫情 監測 與 邊境 管制 措施 國外 入境 後 發燒 咳嗽 適症 狀 請 撥 1922 專線 0800 001922 並依 指示 配戴 口罩 儘 速 醫 同時 主動 告知 醫師 旅遊史 接觸史 以利 及時 診斷 通報'

In [124]:
num_features = 1000

In [125]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(analyzer='word', max_features=num_features,  lowercase = False)
vec.fit(words)
from sklearn.naive_bayes import MultinomialNB
classifier_BOW = MultinomialNB()
classifier_BOW.fit(vec.transform(words), y_train)
print('training set : ',classifier_BOW.score(vec.transform(words), y_train))
print('testing  set : ',classifier_BOW.score(vec.transform(test_words), y_test))

training set :  0.8074534161490683
testing  set :  0.4927536231884058


In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', max_features=num_features,  lowercase = False)
vectorizer.fit(words)
from sklearn.naive_bayes import MultinomialNB
classifier_tfidf = MultinomialNB()
classifier_tfidf.fit(vectorizer.transform(words), y_train)
print('training set : ',classifier_tfidf.score(vectorizer.transform(words), y_train))
print('testing  set : ',classifier_tfidf.score(vectorizer.transform(test_words), y_test))

training set :  0.8012422360248447
testing  set :  0.34782608695652173


In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils import np_utils
np.random.seed(10)
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from keras.regularizers import l2

In [128]:
train_label_onehot = np_utils.to_categorical(y_train)
test_label_onehot = np_utils.to_categorical(y_test)

In [129]:
train_data = vectorizer.transform(words).toarray()
test_data = vectorizer.transform(test_words).toarray()

In [130]:
for i in range(len(train_data[0])):
    print(train_data[0][i])

0.0
0.04946980580233904
0.048987160764365045
0.04946980580233904
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3847060571657662
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.03980703100577878
0.07737424702435064
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.05151676350338941
0.0
0.0
0.0
0.0
0.04946980580233904
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.04851512417329964
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.059958060689308895
0.0
0.0
0.0
0.0
0.0
0.03949800696452279
0.0
0.05769525393188592
0.0
0.04946980580233904
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.07406358365300487
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.04946980580233904
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0860823401918

In [131]:
test_data.shape
test_label_onehot.shape

(69, 14)

In [132]:

j = 600
#for i in range(5,50,50):
model = Sequential()
    

#輸入層：7, 隱藏層：50，輸出層：2
model.add(Dense(units=800,input_dim=num_features,kernel_regularizer=l2(0.01),activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(units=600,kernel_regularizer=l2(0.001),activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(units=400,kernel_regularizer=l2(0.01),activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=50,kernel_regularizer=l2(0.01),activation='relu'))    
model.add(Dense(units=14,kernel_initializer='normal',activation='softmax'))

#定義訓練方式
opt = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])
train_history =model.fit(x=train_data,
                         y=train_label_onehot,
                         validation_split=0.25, 
                         epochs=50000,
                         batch_size=10,
                         verbose=2)

#評估準確率
#scores = model.evaluate(test_data, test_label_onehot)


Train on 120 samples, validate on 41 samples
Epoch 1/50000
 - 0s - loss: 17.8654 - accuracy: 0.0750 - val_loss: 17.8207 - val_accuracy: 0.0488
Epoch 2/50000
 - 0s - loss: 17.7828 - accuracy: 0.0750 - val_loss: 17.7381 - val_accuracy: 0.0488
Epoch 3/50000
 - 0s - loss: 17.7000 - accuracy: 0.1000 - val_loss: 17.6558 - val_accuracy: 0.0488
Epoch 4/50000
 - 0s - loss: 17.6180 - accuracy: 0.0833 - val_loss: 17.5740 - val_accuracy: 0.0488
Epoch 5/50000
 - 0s - loss: 17.5363 - accuracy: 0.0833 - val_loss: 17.4926 - val_accuracy: 0.0488
Epoch 6/50000
 - 0s - loss: 17.4551 - accuracy: 0.0917 - val_loss: 17.4116 - val_accuracy: 0.0488
Epoch 7/50000
 - 0s - loss: 17.3740 - accuracy: 0.0917 - val_loss: 17.3311 - val_accuracy: 0.0488
Epoch 8/50000
 - 0s - loss: 17.2936 - accuracy: 0.1083 - val_loss: 17.2509 - val_accuracy: 0.0488
Epoch 9/50000
 - 0s - loss: 17.2136 - accuracy: 0.0833 - val_loss: 17.1712 - val_accuracy: 0.0488
Epoch 10/50000
 - 0s - loss: 17.1339 - accuracy: 0.1167 - val_loss: 17.09

Epoch 84/50000
 - 0s - loss: 12.2435 - accuracy: 0.2333 - val_loss: 12.2192 - val_accuracy: 0.0732
Epoch 85/50000
 - 0s - loss: 12.1891 - accuracy: 0.2417 - val_loss: 12.1650 - val_accuracy: 0.0732
Epoch 86/50000
 - 0s - loss: 12.1351 - accuracy: 0.2250 - val_loss: 12.1111 - val_accuracy: 0.0732
Epoch 87/50000
 - 0s - loss: 12.0811 - accuracy: 0.2417 - val_loss: 12.0574 - val_accuracy: 0.0732
Epoch 88/50000
 - 0s - loss: 12.0274 - accuracy: 0.2250 - val_loss: 12.0040 - val_accuracy: 0.0732
Epoch 89/50000
 - 0s - loss: 11.9741 - accuracy: 0.2083 - val_loss: 11.9509 - val_accuracy: 0.0732
Epoch 90/50000
 - 0s - loss: 11.9211 - accuracy: 0.2333 - val_loss: 11.8980 - val_accuracy: 0.0732
Epoch 91/50000
 - 0s - loss: 11.8683 - accuracy: 0.2333 - val_loss: 11.8454 - val_accuracy: 0.0732
Epoch 92/50000
 - 0s - loss: 11.8157 - accuracy: 0.2333 - val_loss: 11.7930 - val_accuracy: 0.0732
Epoch 93/50000
 - 0s - loss: 11.7635 - accuracy: 0.2417 - val_loss: 11.7409 - val_accuracy: 0.0732
Epoch 94/5

Epoch 167/50000
 - 0s - loss: 8.5322 - accuracy: 0.2000 - val_loss: 8.5200 - val_accuracy: 0.0244
Epoch 168/50000
 - 0s - loss: 8.4962 - accuracy: 0.2000 - val_loss: 8.4841 - val_accuracy: 0.0488
Epoch 169/50000
 - 0s - loss: 8.4603 - accuracy: 0.2000 - val_loss: 8.4484 - val_accuracy: 0.0488
Epoch 170/50000
 - 0s - loss: 8.4246 - accuracy: 0.1917 - val_loss: 8.4128 - val_accuracy: 0.0488
Epoch 171/50000
 - 0s - loss: 8.3893 - accuracy: 0.1917 - val_loss: 8.3774 - val_accuracy: 0.0488
Epoch 172/50000
 - 0s - loss: 8.3540 - accuracy: 0.1833 - val_loss: 8.3422 - val_accuracy: 0.0488
Epoch 173/50000
 - 0s - loss: 8.3190 - accuracy: 0.1750 - val_loss: 8.3072 - val_accuracy: 0.0488
Epoch 174/50000
 - 0s - loss: 8.2838 - accuracy: 0.2000 - val_loss: 8.2723 - val_accuracy: 0.0488
Epoch 175/50000
 - 0s - loss: 8.2490 - accuracy: 0.1833 - val_loss: 8.2377 - val_accuracy: 0.0488
Epoch 176/50000
 - 0s - loss: 8.2143 - accuracy: 0.1917 - val_loss: 8.2032 - val_accuracy: 0.0488
Epoch 177/50000
 - 0

Epoch 251/50000
 - 0s - loss: 6.0677 - accuracy: 0.1750 - val_loss: 6.0627 - val_accuracy: 0.0732
Epoch 252/50000
 - 0s - loss: 6.0444 - accuracy: 0.1750 - val_loss: 6.0395 - val_accuracy: 0.0732
Epoch 253/50000
 - 0s - loss: 6.0212 - accuracy: 0.1833 - val_loss: 6.0165 - val_accuracy: 0.0732
Epoch 254/50000
 - 0s - loss: 5.9983 - accuracy: 0.1750 - val_loss: 5.9935 - val_accuracy: 0.0732
Epoch 255/50000
 - 0s - loss: 5.9753 - accuracy: 0.1667 - val_loss: 5.9707 - val_accuracy: 0.0732
Epoch 256/50000
 - 0s - loss: 5.9526 - accuracy: 0.1667 - val_loss: 5.9480 - val_accuracy: 0.0732
Epoch 257/50000
 - 0s - loss: 5.9300 - accuracy: 0.1583 - val_loss: 5.9255 - val_accuracy: 0.0732
Epoch 258/50000
 - 0s - loss: 5.9075 - accuracy: 0.1750 - val_loss: 5.9030 - val_accuracy: 0.0732
Epoch 259/50000
 - 0s - loss: 5.8852 - accuracy: 0.1750 - val_loss: 5.8807 - val_accuracy: 0.0732
Epoch 260/50000
 - 0s - loss: 5.8630 - accuracy: 0.1667 - val_loss: 5.8585 - val_accuracy: 0.0732
Epoch 261/50000
 - 0

Epoch 335/50000
 - 0s - loss: 4.5086 - accuracy: 0.1333 - val_loss: 4.5084 - val_accuracy: 0.0732
Epoch 336/50000
 - 0s - loss: 4.4944 - accuracy: 0.1250 - val_loss: 4.4942 - val_accuracy: 0.0732
Epoch 337/50000
 - 0s - loss: 4.4803 - accuracy: 0.1583 - val_loss: 4.4800 - val_accuracy: 0.0732
Epoch 338/50000
 - 0s - loss: 4.4662 - accuracy: 0.1333 - val_loss: 4.4660 - val_accuracy: 0.0732
Epoch 339/50000
 - 0s - loss: 4.4521 - accuracy: 0.1333 - val_loss: 4.4520 - val_accuracy: 0.0732
Epoch 340/50000
 - 0s - loss: 4.4382 - accuracy: 0.1333 - val_loss: 4.4381 - val_accuracy: 0.0732
Epoch 341/50000
 - 0s - loss: 4.4243 - accuracy: 0.1417 - val_loss: 4.4243 - val_accuracy: 0.0732
Epoch 342/50000
 - 0s - loss: 4.4106 - accuracy: 0.1333 - val_loss: 4.4106 - val_accuracy: 0.0732
Epoch 343/50000
 - 0s - loss: 4.3969 - accuracy: 0.1250 - val_loss: 4.3969 - val_accuracy: 0.0732
Epoch 344/50000
 - 0s - loss: 4.3833 - accuracy: 0.1417 - val_loss: 4.3834 - val_accuracy: 0.0732
Epoch 345/50000
 - 0

...

Epoch 9722/50000
 - 0s - loss: 2.6172 - accuracy: 0.0833 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9723/50000
 - 0s - loss: 2.6174 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9724/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9725/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9726/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9727/50000
 - 0s - loss: 2.6174 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9728/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9729/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6674 - val_accuracy: 0.0488
Epoch 9730/50000
 - 0s - loss: 2.6175 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9731/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9732

Epoch 9805/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9806/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9807/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9808/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9809/50000
 - 0s - loss: 2.6174 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9810/50000
 - 0s - loss: 2.6171 - accuracy: 0.0917 - val_loss: 2.6677 - val_accuracy: 0.0488
Epoch 9811/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9812/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6674 - val_accuracy: 0.0488
Epoch 9813/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9814/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9815

Epoch 9888/50000
 - 0s - loss: 2.6169 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9889/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6677 - val_accuracy: 0.0488
Epoch 9890/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6677 - val_accuracy: 0.0488
Epoch 9891/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6678 - val_accuracy: 0.0488
Epoch 9892/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6678 - val_accuracy: 0.0488
Epoch 9893/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6678 - val_accuracy: 0.0488
Epoch 9894/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9895/50000
 - 0s - loss: 2.6171 - accuracy: 0.0917 - val_loss: 2.6678 - val_accuracy: 0.0488
Epoch 9896/50000
 - 0s - loss: 2.6170 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9897/50000
 - 0s - loss: 2.6172 - accuracy: 0.1000 - val_loss: 2.6675 - val_accuracy: 0.0488
Epoch 9898

Epoch 9971/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6678 - val_accuracy: 0.0488
Epoch 9972/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6677 - val_accuracy: 0.0488
Epoch 9973/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6678 - val_accuracy: 0.0488
Epoch 9974/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6677 - val_accuracy: 0.0488
Epoch 9975/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9976/50000
 - 0s - loss: 2.6172 - accuracy: 0.0917 - val_loss: 2.6677 - val_accuracy: 0.0488
Epoch 9977/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9978/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9979/50000
 - 0s - loss: 2.6173 - accuracy: 0.0917 - val_loss: 2.6677 - val_accuracy: 0.0488
Epoch 9980/50000
 - 0s - loss: 2.6171 - accuracy: 0.0917 - val_loss: 2.6676 - val_accuracy: 0.0488
Epoch 9981

KeyboardInterrupt: 